In [60]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [55]:
n_train = 10000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print(vec_length)

252


### Standard autoencoder

Design the network

In [114]:
encoding_dim = 50
input_reg = keras.Input(shape=(vec_length,))
# encoded = layers.Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_reg)
encoded = layers.Dense(vec_length, activation='relu')(input_reg)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
decoded = layers.Dense(encoding_dim, activation='relu')(encoded)
decoded = layers.Dense(vec_length, activation='relu')(decoded)
decoded = layers.Dense(vec_length, activation='sigmoid')(decoded)

autoencoder = keras.Model(input_reg, decoded)
encoder = keras.Model(input_reg, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-3](encoded_input)
decoder_layer = autoencoder.layers[-2](decoder_layer)
decoder_layer = autoencoder.layers[-1](decoder_layer)
decoder = keras.Model(encoded_input, decoder_layer)

Do some learning

In [115]:
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
autoencoder.fit(train_vecs, train_vecs,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(test_vecs, test_vecs))

Epoch 1/50
40/40 [==============================] - 1s 26ms/step - loss: 36.0875 - val_loss: 34.4730
Epoch 2/50
40/40 [==============================] - 1s 19ms/step - loss: 34.2329 - val_loss: 33.7668
Epoch 3/50
40/40 [==============================] - 1s 24ms/step - loss: 32.8164 - val_loss: 31.6981
Epoch 4/50
40/40 [==============================] - 1s 30ms/step - loss: 30.6505 - val_loss: 29.9031
Epoch 5/50
40/40 [==============================] - 1s 21ms/step - loss: 29.3841 - val_loss: 29.2426
Epoch 6/50
40/40 [==============================] - 1s 23ms/step - loss: 28.6721 - val_loss: 28.5643
Epoch 7/50
40/40 [==============================] - 1s 20ms/step - loss: 27.9728 - val_loss: 27.9607
Epoch 8/50
40/40 [==============================] - 1s 26ms/step - loss: 27.0705 - val_loss: 26.7962
Epoch 9/50
40/40 [==============================] - 1s 24ms/step - loss: 25.7012 - val_loss: 25.6405
Epoch 10/50
40/40 [==============================] - 1s 23ms/step - loss: 24.7501 - val_los

In [116]:
encoded_regs = encoder.predict(test_vecs)
decoded_regs = decoder.predict(encoded_regs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['AN45FVI', 'LE23DCR', 'HA99JWO', 'EQ58XKI', 'RP49FNK', 'DJ61XIM', 'WO62NON', 'EA42JCE', 'DV47KTZ', 'HM70DFU']
['AN45FVI', 'LE23DCR', 'HA99JWO', 'EQ58XKI', 'RP49FNK', 'DJ61XIM', 'WO62NON', 'EA42JCE', 'DV47KTZ', 'HM70DFU']
accuracy = 0.999
